In [6]:
# INICIAR
from main import main
from PDconfig import *
from frames.archivosDF import *

main()
df = abrirDF(MeLi)

In [13]:
# FILTRADO DE DATOS GENERAL
df.drop_duplicates(subset='id', keep='last', inplace=True, ignore_index=False)
df.dropna(how="all", subset=['terrenoTotal', 'terrenoEdificado'], inplace=True)

df['fechaUltimaActualizacion'] = pd.to_datetime(df['fechaUltimaActualizacion'], format='%d-%m-%Y')
df['fechaUltimaActualizacion'] = df['fechaUltimaActualizacion'].dt.strftime('%d-%m-%Y')
df.sort_values(by='fechaUltimaActualizacion', ascending=False, inplace=True)
df.drop_duplicates(subset=['coordX', 'coordY', 'precioUSD'], inplace=True, ignore_index=True)

# ELIMINAR TIPO PROP INSERVIBLES
filt = (df["tipoPropiedad"] == "CASA") | (df["tipoPropiedad"] == "DEPARTAMENTO") | (df["tipoPropiedad"] == "TERRENO")
df = df.loc[filt]

# ELIMINAR DATOS SIN CAMPOS RELEVANTES
filtro_T = (df['terrenoEdificado'] == 1)
filtro_P = ((df['precioUSD'] == 1))
df.drop(df[filtro_T | filtro_P].index, inplace=True)
df.drop(df[df['ciudad'] == 'CAPITAL FEDERAL'].index, inplace=True)

# ELIMINAR DATOS SIN VALOR VERDADERO
import numpy as np
filt = df['barrio'].str.contains('INMUEBLES', na=False)
df.loc[filt, 'barrio'] = np.nan

departamentos = ['CALAMUCHITA', 'COLON', 'CRUZ DEL EJE', 
                 'GENERAL ROCA', 'GENERAL SAN MARTIN', 'ISCHILIN', 'JUAREZ CELMAN',
                 'MARCOS JUAREZ', 'MINAS', 'POCHO', 'PUNILLA', 'RIO CUARTO', 'RIO PRIMERO',
                 'RIO SECO', 'RIO SEGUNDO', 'SAN ALBERTO', 'SAN JAVIER', 'SAN JUSTO', 
                 'SANTA MARIA', 'TERCERO ARRIBA', 'TOTORAL', 'TULUMBA', 'UNION']
filt = df['ciudad'].isin(departamentos)
df.loc[filt, 'ciudad'] = df.loc[filt, 'barrio']
filt = (df['ciudad'] == df['barrio'])
df.loc[filt, 'barrio'] = np.nan

In [14]:
# SEPARAR y FILTRAR TIPOS PROPIEDADES
terrenos = df.loc[df["tipoPropiedad"] == "TERRENO"]
casas = df.loc[df["tipoPropiedad"] == "CASA"]
deptos = df.loc[df["tipoPropiedad"] == "DEPARTAMENTO"]
df['tipoPropiedad'].value_counts()

tipoPropiedad
CASA            8941
DEPARTAMENTO    5377
TERRENO         5124
Name: count, dtype: int64